In [ ]:
import numpy as np
import re
from typing import List, Tuple, Dict, Union, Optional, Any

class EinopsError(ValueError):
    pass

# Helper function to parse one side of the pattern (e.g., 'b (h w) c ...')
def _parse_pattern_side(pattern_side: str, axes_lengths: Dict[str, int]) -> Tuple[List[Union[str, Tuple[str, ...], type(...)]], Dict[str, int], List[str]]:
    """
    Parses one side of the einops pattern string.

    Args:
        pattern_side: The string pattern for one side (e.g., 'b (h w) c ...').
        axes_lengths: Dictionary mapping known axis names to their lengths.

    Returns:
        A tuple containing:
        - parsed_components: A list representing the structure (e.g., ['b', ('h', 'w'), 'c', Ellipsis]).
        - component_dims: A dictionary mapping component names (or tuple names) to their calculated dimensions.
        - elementary_axes: A flat list of all elementary axes in the order they appear.

    Raises:
        EinopsError: If the pattern syntax is invalid.
    """
    components = []
    # Use regex to find identifiers, parentheses groups, or ellipsis
    # Valid identifiers: alphanumeric strings (like 'batch', 'height1')
    # Parentheses groups: '(h w)'
    raw_components = re.findall(r"(\b\w+\b|\([^)]+\)|\.\.\.)", pattern_side)

    # Validating that the regex captured the whole pattern correctly
    processed_pattern = "".join(raw_components).replace(" ", "")
    original_pattern = pattern_side.replace(" ", "")
    if processed_pattern != original_pattern:
        diff_index = -1
        min_len = min(len(processed_pattern), len(original_pattern))
        for i in range(min_len):
            if processed_pattern[i] != original_pattern[i]:
                diff_index = i
                break
        if diff_index == -1 and len(original_pattern) != len(processed_pattern):
             diff_index = min_len

        if diff_index != -1:
             start = max(0, diff_index - 5)
             end = min(len(original_pattern), diff_index + 5)
             context = original_pattern[start:end]
             pointer = " " * (diff_index - start) + "^"
             raise EinopsError(f"Invalid pattern syntax near '{context}' (at index {diff_index}) in '{pattern_side}'.\n{pointer}")
        else:
             raise EinopsError(f"Invalid pattern syntax: '{pattern_side}'")


    elementary_axes = []
    component_dims = {}
    ellipsis_found = False

    for comp in raw_components:
        if comp == "...":
            if ellipsis_found:
                raise EinopsError("Ellipsis (...) can appear at most once per side.")
            ellipsis_found = True
            components.append(...)
            elementary_axes.append(...)
        elif comp.startswith("(") and comp.endswith(")"):
            axes_in_group = tuple(comp[1:-1].split())
            if not all(re.match(r"^\w+$", ax) for ax in axes_in_group):
                 raise EinopsError(f"Invalid axis names inside parentheses: '{comp}'")
            if not axes_in_group:
                 raise EinopsError(f"Empty parentheses found: '{comp}'")
            components.append(axes_in_group)
            dim = 1
            known = True
            for ax in axes_in_group:
                if ax in axes_lengths:
                    dim *= axes_lengths[ax]
                elif ax == '1':
                    dim *= 1
                else:
                    known = False
                    break
            if known:
                component_dims[axes_in_group] = dim
            elementary_axes.extend(axes_in_group)
        elif re.match(r"^\w+$", comp):
            components.append(comp)
            if comp in axes_lengths:
                component_dims[comp] = axes_lengths[comp]
            elif comp == '1':
                 component_dims[comp] = 1
            elementary_axes.append(comp)
        else:
            raise EinopsError(f"Unknown component format in pattern: '{comp}'")

    # Check for duplicate elementary axes *within this side*
    seen_axes = set()
    for ax in elementary_axes:
        if ax != ... and ax != '1':
            if ax in seen_axes:
                raise EinopsError(f"Axis '{ax}' appears multiple times on the same side of the pattern: '{pattern_side}'")
            seen_axes.add(ax)

    return components, component_dims, elementary_axes


def rearrange(tensor: np.ndarray, pattern: str, **axes_lengths: int) -> np.ndarray:
    """
    Replicates the core functionality of einops.rearrange.

    Supports reshaping, transposition, splitting, merging, and repeating axes.

    Args:
        tensor: The input NumPy array.
        pattern: The einops pattern string (e.g., 'b h w c -> b c h w', '(h w) c -> h w c').
        **axes_lengths: Keyword arguments specifying the lengths of newly created axes
                         (e.g., h=3, w=4 for splitting).

    Returns:
        The rearranged NumPy array.

    Raises:
        EinopsError: For invalid patterns, shape mismatches, or incorrect axes_lengths.
    """
    # --- 1. Parse the pattern ---
    if "->" not in pattern:
        raise EinopsError("Pattern must include '->' separator.")
    left_str, right_str = pattern.split("->", 1)
    left_str = left_str.strip()
    right_str = right_str.strip()

    if not left_str:
        raise EinopsError("Left side of the pattern cannot be empty.")
    if not right_str:
         raise EinopsError("Right side of the pattern cannot be empty.")

    known_axes = axes_lengths.copy()

    left_parsed, left_dims, left_elementary_raw = _parse_pattern_side(left_str, known_axes)
    right_parsed, _, _ = _parse_pattern_side(right_str, known_axes) # Dims/elementary might be incomplete

    # --- 2. Analyze Input Pattern & Tensor Shape ---
    input_shape = tensor.shape
    inferred_axes = {}
    ellipsis_present_in_left = ... in left_parsed
    ellipsis_axes_names = []
    ellipsis_shape = ()
    left_elementary = list(left_elementary_raw) # Making a copy to modify

    if ellipsis_present_in_left:
        ellipsis_pos = left_parsed.index(...)
        non_ellipsis_dims = len(left_parsed) - 1
        if non_ellipsis_dims > len(input_shape):
            raise EinopsError(f"Input tensor has {len(input_shape)} dimensions, but pattern '{left_str}' requires at least {non_ellipsis_dims} non-ellipsis dimensions.")

        ellipsis_len = len(input_shape) - non_ellipsis_dims
        if ellipsis_len < 0:
            raise EinopsError(f"Internal error: Negative ellipsis length calculated.")

        ellipsis_shape = input_shape[ellipsis_pos : ellipsis_pos + ellipsis_len]
        ellipsis_axes_names = [f"_ellipsis_{i}" for i in range(ellipsis_len)]

        # Update left_elementary list to include specific ellipsis axes names
        temp_left_elementary = []
        for ax in left_elementary:
            if ax == ...:
                temp_left_elementary.extend(ellipsis_axes_names)
            else:
                temp_left_elementary.append(ax)
        left_elementary = temp_left_elementary

        for i, dim_size in enumerate(ellipsis_shape):
             known_axes[ellipsis_axes_names[i]] = dim_size
             inferred_axes[ellipsis_axes_names[i]] = dim_size

    else: # No ellipsis
        if len(left_parsed) != len(input_shape):
            raise EinopsError(f"Input tensor has {len(input_shape)} dimensions, but pattern '{left_str}' has {len(left_parsed)} components.")

    # Match components to input shape and infer/validate dimensions
    final_input_shape_map = {} # Mapping elementary axis name (incl. internal) -> its size
    idx_tensor = 0
    for component in left_parsed:
        if component == ...:
            for i, dim_size in enumerate(ellipsis_shape):
                 final_input_shape_map[ellipsis_axes_names[i]] = dim_size
            idx_tensor += len(ellipsis_shape)
            continue

        if idx_tensor >= len(input_shape):
             raise EinopsError(f"Pattern component '{component}' at index {idx_tensor} exceeds tensor dimensions ({len(input_shape)}). Pattern: '{left_str}'")

        current_dim_size = input_shape[idx_tensor]

        if isinstance(component, str): # Single axis like 'b' or '1'
            axis_name = component
            if axis_name == '1':
                if current_dim_size != 1:
                     raise EinopsError(f"Pattern expects dimension size 1 for component '{axis_name}' at index {idx_tensor}, but tensor has {current_dim_size}.")
                internal_axis_name = f"_literal_1_{idx_tensor}"
                final_input_shape_map[internal_axis_name] = 1
            elif axis_name in known_axes:
                if known_axes[axis_name] != current_dim_size:
                    raise EinopsError(f"Dimension mismatch for axis '{axis_name}'. Pattern/axes_lengths expects {known_axes[axis_name]}, but tensor has {current_dim_size} at index {idx_tensor}.")
                final_input_shape_map[axis_name] = current_dim_size
            else: # Infer dimension
                known_axes[axis_name] = current_dim_size
                inferred_axes[axis_name] = current_dim_size
                final_input_shape_map[axis_name] = current_dim_size

        elif isinstance(component, tuple): # Split/Merged group like '(h w)'
            axes_in_group = component
            unknown_axes = [ax for ax in axes_in_group if ax not in known_axes and ax != '1']

            if len(unknown_axes) > 1:
                raise EinopsError(f"Cannot infer dimensions for multiple unknown axes {unknown_axes} in group '{component}' for dimension size {current_dim_size}. Please provide lengths via **axes_lengths.")
            elif len(unknown_axes) == 1:
                unknown_ax = unknown_axes[0]
                product_known = 1
                for ax in axes_in_group:
                    if ax != unknown_ax:
                        product_known *= known_axes.get(ax, 1)

                if product_known == 0:
                    if current_dim_size == 0: inferred_len = 0
                    else: raise EinopsError(f"Inconsistent state: Known axes product is 0, but dimension size is {current_dim_size} for group '{component}'.")
                elif current_dim_size % product_known != 0:
                    raise EinopsError(f"Dimension {current_dim_size} at index {idx_tensor} cannot be split according to pattern '{component}' with known lengths. {current_dim_size} is not divisible by {product_known}.")
                else:
                    inferred_len = current_dim_size // product_known

                known_axes[unknown_ax] = inferred_len
                inferred_axes[unknown_ax] = inferred_len

            product_total = 1
            for ax in axes_in_group:
                 product_total *= known_axes.get(ax, 1)

            if product_total != current_dim_size:
                 raise EinopsError(f"Dimension mismatch for group '{component}'. Product of lengths ({product_total}) does not match tensor dimension {current_dim_size} at index {idx_tensor}.")

            for ax in axes_in_group:
                 if ax == '1':
                     internal_ax_name = f"_literal_1_in_group_{idx_tensor}_{ax}"
                     final_input_shape_map[internal_ax_name] = 1
                 else:
                     final_input_shape_map[ax] = known_axes.get(ax)
        idx_tensor += 1

    # --- 3. Determine Intermediate Permutation and Reshape ---
    reshape_shape_for_split = []
    source_axes_order_after_split = []
    current_original_idx = 0
    for component in left_parsed:
        if component == ...:
            reshape_shape_for_split.extend(ellipsis_shape)
            source_axes_order_after_split.extend(ellipsis_axes_names)
            current_original_idx += len(ellipsis_shape)
        elif isinstance(component, str):
            axis_name = component
            reshape_shape_for_split.append(input_shape[current_original_idx])
            if axis_name == '1':
                 source_axes_order_after_split.append(f"_literal_1_{current_original_idx}")
            else:
                 source_axes_order_after_split.append(axis_name)
            current_original_idx += 1
        elif isinstance(component, tuple):
            split_dims = []
            axes_names_in_group = []
            for ax in component:
                dim_val = known_axes.get(ax, 1)
                split_dims.append(dim_val)
                if ax == '1':
                    axes_names_in_group.append(f"_literal_1_in_group_{current_original_idx}_{ax}")
                else:
                    axes_names_in_group.append(ax)
            reshape_shape_for_split.extend(split_dims)
            source_axes_order_after_split.extend(axes_names_in_group)
            current_original_idx += 1

    try:
        if not reshape_shape_for_split and tensor.ndim == 0:
             intermediate_tensor = tensor
        elif np.prod(tensor.shape, dtype=np.int64) == np.prod(reshape_shape_for_split, dtype=np.int64):
             intermediate_tensor = tensor.reshape(reshape_shape_for_split)
        else:
            raise ValueError(f"Total size mismatch: input {np.prod(tensor.shape, dtype=np.int64)} vs reshape {np.prod(reshape_shape_for_split, dtype=np.int64)}")
    except (ValueError, TypeError) as e:
         raise EinopsError(f"Cannot reshape tensor with shape {tensor.shape} to {reshape_shape_for_split} based on pattern '{left_str}'. Original error: {e}")

    # Map original elementary axes (incl '1') to their internal names for target order
    target_axes_order_intermediate = []
    temp_idx_map_literal_1 = {i: f"_literal_1_{i}" for i, comp in enumerate(left_parsed) if isinstance(comp, str) and comp == '1'}
    temp_idx_map_group_1 = {}
    for i, comp in enumerate(left_parsed):
        if isinstance(comp, tuple):
            for ax_in_group in comp:
                if ax_in_group == '1':
                    temp_idx_map_group_1[(i, ax_in_group)] = f"_literal_1_in_group_{i}_{ax_in_group}"

    original_elementary_axes = []
    for comp in left_parsed:
         if comp == ...: original_elementary_axes.extend(ellipsis_axes_names)
         elif isinstance(comp, str): original_elementary_axes.append(comp)
         elif isinstance(comp, tuple): original_elementary_axes.extend(list(comp))

    for ax_or_ellipsis in original_elementary_axes:
        component_idx = -1; component_origin = None
        temp_scan_idx = 0
        for idx, comp in enumerate(left_parsed):
            num_dims_in_comp = 0
            if comp == ...:
                num_dims_in_comp = len(ellipsis_axes_names)
                if ax_or_ellipsis in ellipsis_axes_names: component_idx, component_origin = idx, comp; break
            elif isinstance(comp, str):
                num_dims_in_comp = 1
                if ax_or_ellipsis == comp: component_idx, component_origin = idx, comp; break
            elif isinstance(comp, tuple):
                num_dims_in_comp = len(comp)
                if ax_or_ellipsis in comp: component_idx, component_origin = idx, comp; break
            temp_scan_idx += num_dims_in_comp
        if component_idx == -1: raise EinopsError(f"Internal error: Could not find origin component for axis '{ax_or_ellipsis}'")

        if ax_or_ellipsis == '1':
            internal_name = None
            if isinstance(component_origin, str): internal_name = temp_idx_map_literal_1.get(component_idx)
            elif isinstance(component_origin, tuple): internal_name = temp_idx_map_group_1.get((component_idx, ax_or_ellipsis))
            if internal_name: target_axes_order_intermediate.append(internal_name)
            else: raise EinopsError(f"Internal error: Failed to find internal name for axis '1' from component {component_idx}.")
        elif ax_or_ellipsis.startswith("_ellipsis_"): target_axes_order_intermediate.append(ax_or_ellipsis)
        else: target_axes_order_intermediate.append(ax_or_ellipsis) # Regular named axis

    # Calculate and apply intermediate permutation
    if intermediate_tensor.ndim > 0:
        try:
            source_indices = {axis: i for i, axis in enumerate(source_axes_order_after_split)}
            permutation = [source_indices[axis] for axis in target_axes_order_intermediate]
            if len(permutation) != intermediate_tensor.ndim: raise ValueError(f"Permutation length {len(permutation)} != tensor ndim {intermediate_tensor.ndim}")
            if len(set(permutation)) != intermediate_tensor.ndim: raise ValueError(f"Permutation indices not unique: {permutation}")
        except (KeyError, ValueError) as e:
             raise EinopsError(f"Internal error during intermediate permutation. Source: {source_axes_order_after_split}, Target: {target_axes_order_intermediate}. Error: {e}")
        intermediate_tensor = intermediate_tensor.transpose(permutation)

    # --- 4. Analyze Output Pattern & Determine Final Operations ---
    final_axes_map = final_input_shape_map.copy()
    repeat_instructions = {}
    ellipsis_present_in_right = ... in right_parsed
    output_ellipsis_axes_names = []
    if ellipsis_present_in_right:
        if not ellipsis_present_in_left: raise EinopsError("Ellipsis (...) present in output pattern but not in input pattern.")
        output_ellipsis_axes_names = ellipsis_axes_names
    elif ellipsis_present_in_left: raise EinopsError("Ellipsis (...) present in input pattern but not specified in output pattern.")

    temp_right_elementary_structure = []
    for comp in right_parsed:
        if comp == ...: temp_right_elementary_structure.extend(output_ellipsis_axes_names)
        elif isinstance(comp, str): temp_right_elementary_structure.append(comp)
        elif isinstance(comp, tuple): temp_right_elementary_structure.extend(list(comp))

    input_elementary_axes_set = set(target_axes_order_intermediate)
    output_elementary_named_axes = set(ax for ax in temp_right_elementary_structure if ax != '1' and not ax.startswith('_ellipsis_'))
    potential_new_axes = output_elementary_named_axes - set(ax for ax in input_elementary_axes_set if not ax.startswith('_'))
    possible_repeat_sources_internal = [ax for ax in target_axes_order_intermediate if ax.startswith('_literal_1')]
    possible_repeat_sources_internal.extend([ax for ax in target_axes_order_intermediate if not ax.startswith('_') and final_axes_map.get(ax) == 1])

    for out_ax in list(potential_new_axes):
        if out_ax in axes_lengths:
            repeat_count = axes_lengths[out_ax]
            if not possible_repeat_sources_internal: raise EinopsError(f"Output axis '{out_ax}' requires repetition (length {repeat_count}), but no available input axis of size 1 found to repeat from {target_axes_order_intermediate}.")
            source_ax_internal = possible_repeat_sources_internal.pop(0)
            repeat_instructions[out_ax] = (source_ax_internal, repeat_count)
            final_axes_map[out_ax] = repeat_count
            potential_new_axes.remove(out_ax)
        else:
             raise EinopsError(f"Axis '{out_ax}' appears only in the output pattern ('{right_str}') but its length is not specified in **axes_lengths.")

    used_repeat_sources_internal = {src for src, count in repeat_instructions.values()}
    required_input_axes_final = set(ax for ax in target_axes_order_intermediate if not ax.startswith('_literal_1') and not ax.startswith('_ellipsis_') and ax not in used_repeat_sources_internal)
    present_output_axes_final = set(ax for ax in temp_right_elementary_structure if ax != '1' and not ax.startswith('_ellipsis_') and ax not in repeat_instructions)
    missing_axes = required_input_axes_final - present_output_axes_final
    if missing_axes: raise EinopsError(f"Input axes {missing_axes} are not present in the output pattern '{right_str}' and not used for repetition.")

    # --- 5. Perform Repetition, Final Permutation and Reshape ---
    current_tensor = intermediate_tensor
    current_axes_order_for_ops = list(target_axes_order_intermediate)
    final_target_axes_order = list(target_axes_order_intermediate)
    source_indices_to_remove_map = {}

    repeat_ops = [{'new': n, 'source': s, 'count': c} for n, (s, c) in repeat_instructions.items()]

    for op in repeat_ops:
        new_axis, source_axis_internal, repeat_count = op['new'], op['source'], op['count']
        try:
            tensor_source_axis_index = current_axes_order_for_ops.index(source_axis_internal)
        except ValueError: raise EinopsError(f"Internal error: Cannot find source axis '{source_axis_internal}' for repeating into '{new_axis}' in current tensor axes {current_axes_order_for_ops}.")

        current_tensor = np.expand_dims(current_tensor, axis=tensor_source_axis_index)
        current_tensor = np.repeat(current_tensor, repeat_count, axis=tensor_source_axis_index)
        current_axes_order_for_ops.insert(tensor_source_axis_index, new_axis)

        final_source_axis_index = final_target_axes_order.index(source_axis_internal)
        final_target_axes_order.insert(final_source_axis_index, new_axis)
        source_indices_to_remove_map[source_axis_internal] = final_source_axis_index + 1

    indices_to_squeeze = sorted(source_indices_to_remove_map.values(), reverse=True)
    for index in indices_to_squeeze:
         current_tensor = np.squeeze(current_tensor, axis=index)
         removed_axis_name = final_target_axes_order.pop(index)

    # Build final target order from right pattern
    output_target_axes_order = []
    available_internal_1s = [ax for ax in final_target_axes_order if ax.startswith('_literal_1')]
    for comp in right_parsed:
        if comp == ...: output_target_axes_order.extend(output_ellipsis_axes_names)
        elif isinstance(comp, str):
            if comp in repeat_instructions: output_target_axes_order.append(comp)
            elif comp == '1':
                 if not available_internal_1s: raise EinopsError("Output pattern requests anonymous axis '1', but no dimension of size 1 remains available.")
                 internal_1_name_to_use = available_internal_1s.pop(0)
                 output_target_axes_order.append(internal_1_name_to_use)
            else: output_target_axes_order.append(comp)
        elif isinstance(comp, tuple): output_target_axes_order.extend(list(comp))

    # Calculate and apply final permutation
    if current_tensor.ndim > 0:
        try:
            source_final_indices = {axis: i for i, axis in enumerate(final_target_axes_order)}
            final_permutation = [source_final_indices[axis] for axis in output_target_axes_order]
            if len(final_permutation) != current_tensor.ndim: raise ValueError(f"Permutation length {len(final_permutation)} != tensor ndim {current_tensor.ndim}")
            if len(set(final_permutation)) != len(final_permutation): raise ValueError(f"Permutation indices not unique: {final_permutation}")
        except (KeyError, ValueError) as e:
             raise EinopsError(f"Internal error during final permutation. Current axes: {final_target_axes_order}, Target: {output_target_axes_order}. Error: {e}")
        permuted_tensor = current_tensor.transpose(final_permutation)
    else: # Scalar case
        permuted_tensor = current_tensor

    # Final reshape for merging
    final_shape = []
    axis_counter_in_permuted = 0
    for component in right_parsed:
        if component == ...:
            ellipsis_dims = [final_axes_map[ax] for ax in output_ellipsis_axes_names]
            final_shape.extend(ellipsis_dims)
            axis_counter_in_permuted += len(ellipsis_dims)
        elif isinstance(component, str):
            if component == '1':
                 if permuted_tensor.ndim > 0 and permuted_tensor.shape[axis_counter_in_permuted] != 1: raise EinopsError(f"Internal error: Expected dimension size 1 for anonymous '1' at index {axis_counter_in_permuted}, got {permuted_tensor.shape[axis_counter_in_permuted]}.")
                 final_shape.append(1)
            else:
                final_shape.append(final_axes_map[component])
            axis_counter_in_permuted += 1
        elif isinstance(component, tuple):
            merged_dim = 1
            num_axes_in_group = 0
            for ax in component:
                 if ax not in final_axes_map: raise EinopsError(f"Axis '{ax}' used in merging group '{component}' not found in final axis map.")
                 merged_dim *= final_axes_map[ax]
                 num_axes_in_group += 1
            final_shape.append(merged_dim)
            axis_counter_in_permuted += num_axes_in_group

    target_size = np.prod(final_shape, dtype=np.int64) if final_shape else 1
    if permuted_tensor.size != target_size:
         raise EinopsError(f"Internal error: Number of elements mismatch before final reshape. Tensor size: {permuted_tensor.size}, Target shape product: {target_size}. Target shape: {final_shape}")

    # Handle reshaping involving scalars
    if permuted_tensor.ndim == 0 and final_shape:
         if target_size == 1: result_tensor = np.full(final_shape, permuted_tensor.item())
         else: raise EinopsError(f"Cannot reshape scalar input to non-scalar shape {final_shape} with size {target_size}")
    elif not final_shape and permuted_tensor.ndim > 0:
         if permuted_tensor.size == 1: result_tensor = permuted_tensor.item()
         else: raise EinopsError(f"Cannot reshape non-scalar tensor with size {permuted_tensor.size} to scalar '()'")
    elif not final_shape and permuted_tensor.ndim == 0:
         result_tensor = permuted_tensor # Scalar to scalar
    else:
         result_tensor = permuted_tensor.reshape(final_shape)

    return result_tensor



In [ ]:

# --- Unit Tests ---
import unittest
import numpy as np

try:
    from einops import rearrange as rearrange_ref
    einops_available = True
except ImportError:
    einops_available = False
    print("Einops library not found. Skipping comparison tests.")
    rearrange_ref = None 

class TestRearrangeScratch(unittest.TestCase):

    def assert_np_equal(self, arr1, arr2):
        np.testing.assert_array_equal(arr1, arr2)

    def test_transpose(self):
        x = np.arange(12).reshape(3, 4)
        expected = x.T
        result = rearrange(x, 'h w -> w h')
        self.assert_np_equal(result, expected)
        if einops_available:
            self.assert_np_equal(result, rearrange_ref(x, 'h w -> w h'))

    def test_split_axis(self):
        x = np.arange(24).reshape(12, 2)
        expected = x.reshape(3, 4, 2)
        result = rearrange(x, '(h w) c -> h w c', h=3)
        self.assert_np_equal(result, expected)
        if einops_available:
             self.assert_np_equal(result, rearrange_ref(x, '(h w) c -> h w c', h=3))
        result_infer = rearrange(x, '(h w) c -> h w c', h=3)
        self.assert_np_equal(result_infer, expected)

    def test_merge_axes(self):
        x = np.arange(24).reshape(2, 3, 4)
        expected = x.reshape(6, 4)
        result = rearrange(x, 'a b c -> (a b) c')
        self.assert_np_equal(result, expected)
        if einops_available:
            self.assert_np_equal(result, rearrange_ref(x, 'a b c -> (a b) c'))

        expected_2 = x.reshape(2, 12)
        result_2 = rearrange(x, 'a b c -> a (b c)')
        self.assert_np_equal(result_2, expected_2)
        if einops_available:
             self.assert_np_equal(result_2, rearrange_ref(x, 'a b c -> a (b c)'))

    def test_split_and_merge(self):
        x = np.arange(120).reshape(12, 10) # (3*4, 2*5)
        expected = x.reshape(3, 4, 2, 5).transpose(0, 2, 1, 3).reshape(6, 20) # h w c1 c2 -> h c1 w c2 -> (h c1) (w c2)
        result = rearrange(x, '(h w) (c1 c2) -> (h c1) (w c2)', h=3, c1=2)
        self.assert_np_equal(result, expected)
        if einops_available:
            self.assert_np_equal(result, rearrange_ref(x, '(h w) (c1 c2) -> (h c1) (w c2)', h=3, c1=2))

    def test_repeat_axis_from_1(self):
        x = np.arange(15).reshape(3, 1, 5)
        expected = np.repeat(x, 4, axis=1) # Repeat the second dimension 4 times
        result = rearrange(x, 'a 1 c -> a b c', b=4)
        self.assert_np_equal(result, expected)
        # Removed comparison with rearrange_ref as it handles this pattern differently

    def test_repeat_axis_from_named_1(self):
        x = np.arange(15).reshape(3, 1, 5)
        expected = np.repeat(x, 4, axis=1)
        result = rearrange(x, 'a one c -> a b c', b=4) # 'one' has inferred size 1
        self.assert_np_equal(result, expected)
        # Removed comparison with rearrange_ref

    # Add test for repeating from a named axis that happens to be size 1
    def test_repeat_axis_from_named_explicit_1(self):
        x = np.arange(15).reshape(3, 1, 5)
        expected = np.repeat(x, 4, axis=1)
        result = rearrange(x, 'a one c -> a b c', one=1, b=4) # provide one=1 explicitly
        self.assert_np_equal(result, expected)
        # Removed comparison with rearrange_ref

    def test_ellipsis_simple_transpose(self):
        x = np.arange(60).reshape(2, 3, 5, 2) # b1 b2 h w
        expected = x.transpose(0, 1, 3, 2)   # b1 b2 w h
        result = rearrange(x, '... h w -> ... w h')
        self.assert_np_equal(result, expected)
        if einops_available:
            self.assert_np_equal(result, rearrange_ref(x, '... h w -> ... w h'))

    def test_ellipsis_middle(self):
         x = np.arange(60).reshape(2, 3, 5, 2) # b h w c
         expected = x.transpose(0, 3, 1, 2)    # b c h w
         result = rearrange(x, 'b ... c -> b c ...')
         self.assert_np_equal(result, expected)
         if einops_available:
              self.assert_np_equal(result, rearrange_ref(x, 'b ... c -> b c ...'))

    def test_ellipsis_split_merge(self):
        x = np.arange(120).reshape(2, 6, 10) # b (h w) (c1 c2)
        expected = x.reshape(2, 2, 3, 5, 2).transpose(0, 1, 3, 2, 4).reshape(2, 10, 6) # b h w c1 c2 -> b h c1 w c2 -> b (h c1) (w c2)
        result = rearrange(x, 'b (h w) (c1 c2) -> b (h c1) (w c2)', h=2, c1=5)
        self.assert_np_equal(result, expected)
        if einops_available:
            self.assert_np_equal(result, rearrange_ref(x, 'b (h w) (c1 c2) -> b (h c1) (w c2)', h=2, c1=5))

        # With ellipsis
        x_el = np.arange(240).reshape(2, 2, 6, 10) # b1 b2 (h w) (c1 c2)
        expected_el = x_el.reshape(2, 2, 2, 3, 5, 2).transpose(0, 1, 2, 4, 3, 5).reshape(2, 2, 10, 6) # ... h w c1 c2 -> ... h c1 w c2 -> ... (h c1) (w c2)
        result_el = rearrange(x_el, '... (h w) (c1 c2) -> ... (h c1) (w c2)', h=2, c1=5)
        self.assert_np_equal(result_el, expected_el)
        if einops_available:
             self.assert_np_equal(result_el, rearrange_ref(x_el, '... (h w) (c1 c2) -> ... (h c1) (w c2)', h=2, c1=5))


    def test_identity(self):
        x = np.arange(24).reshape(2, 3, 4)
        result = rearrange(x, 'a b c -> a b c')
        self.assert_np_equal(result, x)
        if einops_available:
            self.assert_np_equal(result, rearrange_ref(x, 'a b c -> a b c'))

    def test_identity_ellipsis(self):
        x = np.arange(120).reshape(2, 3, 4, 5)
        result = rearrange(x, '... -> ...')
        self.assert_np_equal(result, x)
        if einops_available:
             self.assert_np_equal(result, rearrange_ref(x, '... -> ...'))

        result2 = rearrange(x, 'a ... c -> a ... c')
        self.assert_np_equal(result2, x)
        if einops_available:
             self.assert_np_equal(result2, rearrange_ref(x, 'a ... c -> a ... c'))

    # --- Error Handling Tests ---

    def test_error_invalid_pattern_syntax(self):
        x = np.zeros((2, 3))
        with self.assertRaisesRegex(EinopsError, "Invalid pattern syntax"):
            rearrange(x, 'a (b -> a b') # Missing closing parenthesis
        with self.assertRaisesRegex(EinopsError, "separator"):
            rearrange(x, 'a b c d')
        with self.assertRaisesRegex(EinopsError, "multiple times"):
            rearrange(x, 'a a -> b c')
        with self.assertRaisesRegex(EinopsError, "multiple times"):
            rearrange(x, 'a b -> c c')
        with self.assertRaisesRegex(EinopsError, "Ellipsis .* can appear at most once"):
            rearrange(x, '... a ... -> a')
        with self.assertRaisesRegex(EinopsError, "Invalid axis names"):
             rearrange(x, '(a b-c) -> a b c')
        with self.assertRaisesRegex(EinopsError, "Invalid pattern syntax near"):
             rearrange(x, 'a b -> a ^ b') # Invalid character


    def test_error_shape_mismatch(self):
        x = np.zeros((2, 3, 4))
        # Test case 1: Too few components in pattern (no ellipsis)
        with self.assertRaisesRegex(EinopsError, "Input tensor has 3 dimensions, but pattern 'a b' has 2 components."):
            rearrange(x, 'a b -> a b')
        # Test case 2: Too many components in pattern (no ellipsis)
        with self.assertRaisesRegex(EinopsError, "Input tensor has 3 dimensions, but pattern 'a b c d' has 4 components."):
            rearrange(x, 'a b c d -> a b c d')
        # Test case 3: Explicit dimension mismatch
        with self.assertRaisesRegex(EinopsError, "Dimension mismatch for axis 'a'"):
            rearrange(x, 'a b c -> a b c', a=5)
        # Test case 4: Expecting size 1 but got different size
        with self.assertRaisesRegex(EinopsError, "Pattern expects dimension size 1"):
             # Input shape[1] is 3, pattern expects '1' at index 1
             rearrange(x, 'a 1 c -> a b c')
        # Test case 5: Too many components in pattern for given tensor dimensions (no ellipsis) - Updated Regex
        with self.assertRaisesRegex(EinopsError, "Input tensor has 2 dimensions, but pattern 'a b c' has 3 components."):
             rearrange(np.zeros((2,3)), 'a b c -> a b c')


    def test_error_split_mismatch(self):
        x = np.zeros((12, 10))
        with self.assertRaisesRegex(EinopsError, "Dimension 12 .* cannot be split"):
            rearrange(x, '(h w) c -> h w c', h=5) # 12 not divisible by 5
        with self.assertRaisesRegex(EinopsError, "Product of lengths .* does not match"):
            rearrange(x, '(h w) c -> h w c', h=3, w=5) # 3*5 != 12

    def test_error_missing_axes_lengths(self):
        x = np.zeros((12, 10))
        with self.assertRaisesRegex(EinopsError, "Cannot infer dimensions for multiple unknown axes"):
            rearrange(x, '(h w) c -> h w c') # Missing h and w
        x_rep = np.zeros((3, 1, 5))
        with self.assertRaisesRegex(EinopsError, "length is not specified"):
             rearrange(x_rep, 'a 1 c -> a b c') # Missing b

    def test_error_axis_usage(self):
         x = np.zeros((2, 3, 4))
         with self.assertRaisesRegex(EinopsError, "Input axes {'c'} are not present"):
              rearrange(x, 'a b c -> a b')
         with self.assertRaisesRegex(EinopsError, "Axis 'd' appears only in the output"):
              rearrange(x, 'a b c -> a b d') # d not defined
         with self.assertRaisesRegex(EinopsError, "Output axis 'b' requires repetition .* no available input axis of size 1"):
              rearrange(x, 'a c d -> a b c d', b=4) # No dim of size 1 in input
         with self.assertRaisesRegex(EinopsError, "Ellipsis .* present in input pattern but not specified in output"):
              rearrange(np.zeros((2,3,4,5)), 'a ... c -> a c')
         with self.assertRaisesRegex(EinopsError, "Ellipsis .* present in output pattern but not in input"):
              rearrange(np.zeros((2,3,4)), 'a b c -> a ... c')



# --- Rearrange Function ---
import unittest
import sys
import os 


try:
    suite = unittest.TestSuite()
    # Use the modern way to load tests
    suite.addTest(unittest.TestLoader().loadTestsFromTestCase(TestRearrangeScratch))
    runner = unittest.TextTestRunner(verbosity=2) # Increase verbosity
    print("--- Running Unit Tests ---")
    test_result = runner.run(suite)
    print("--- Unit Tests Finished ---")
    # Print summary
    print(f"\nTests Run: {test_result.testsRun}")
    print(f"Errors: {len(test_result.errors)}")
    print(f"Failures: {len(test_result.failures)}")
    if not test_result.wasSuccessful():
         print("\n--- Errors ---")
         for test, err in test_result.errors:
              print(f"Test: {test}\nError: {err}\n")
         print("\n--- Failures ---")
         for test, err in test_result.failures:
              print(f"Test: {test}\nFailure: {err}\n")

finally:
    pass


test_ellipsis_middle (__main__.TestRearrangeScratch.test_ellipsis_middle) ... ok
test_ellipsis_simple_transpose (__main__.TestRearrangeScratch.test_ellipsis_simple_transpose) ... ok
test_ellipsis_split_merge (__main__.TestRearrangeScratch.test_ellipsis_split_merge) ... ok
test_error_axis_usage (__main__.TestRearrangeScratch.test_error_axis_usage) ... ok
test_error_invalid_pattern_syntax (__main__.TestRearrangeScratch.test_error_invalid_pattern_syntax) ... ok
test_error_missing_axes_lengths (__main__.TestRearrangeScratch.test_error_missing_axes_lengths) ... ok
test_error_shape_mismatch (__main__.TestRearrangeScratch.test_error_shape_mismatch) ... ok
test_error_split_mismatch (__main__.TestRearrangeScratch.test_error_split_mismatch) ... ok
test_identity (__main__.TestRearrangeScratch.test_identity) ... ok
test_identity_ellipsis (__main__.TestRearrangeScratch.test_identity_ellipsis) ... ok
test_merge_axes (__main__.TestRearrangeScratch.test_merge_axes) ... ok
test_repeat_axis_from_1 (__ma

--- Running Unit Tests ---
--- Unit Tests Finished ---

Tests Run: 17
Errors: 0
Failures: 0
